In [1]:
import json
import requests
from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'requests'

### Imports

2. Sent http request

In [ ]:
url = "https://www.ceneo.pl/126710499"
response = requests.get(url);

3. Checking the code of http response

In [ ]:
response.status

4. Parse the html code of first page wth opinions

In [ ]:
page_dom = BeautifulSoup(response.text,"html_parse")
print(page_dom.prettify())

5. Extract required data from the page

In [ ]:
opinion = page_dom.select_one("div.js_product-review")
print(type(page_dom))
print(type(opinion))
print(opinion.prettify())